Goal is to calculate optimal picks based on win probability, public pick probability, and number of players in pool. Idea would be: the more players in the pool, the riskier you have to be to increase probability of winning pool.

In [3]:
import pandas as pd
import numpy as np
import pickle

In [378]:
#model = pickle.load(open('data/game_prediction_model.pickle', 'rb'))
#team_cols = pickle.load(open('data/game_prediction_model_covariates.pickle', 'rb'))
starting_teams = pickle.load(open('data/starting_teams.pickle', 'rb'))

In [380]:
df_predictions = pd.read_csv('data/2019_tourney_predictions.csv')
df_predictions = df_predictions[['RoundOf32','SweetSixteen','EliteEight',\
                                'FinalFour','Championship','Champions']]
df_predictions.columns = ['R1','R2','R3','R4','R5','R6']
df_predictions.head()

,R1,R2,R3,R4,R5,R6
0,0.78,0.61,0.36,0.23,0.13,0.06
1,0.22,0.02,0.00,0.00,0.00,0.00
2,0.44,0.15,0.03,0.02,0.00,0.00
3,0.56,0.22,0.07,0.03,0.01,0.00
4,0.71,0.42,0.21,0.07,0.02,0.01


In [343]:
def _generate_bracket(df_predictions):
    df_teams = df_predictions.copy(deep=True)
    rounds = len(df_teams.columns)
    df_teams = df_teams.cumsum()
    df_bracket = pd.DataFrame(columns=df_teams.columns, index=df_teams.index)
    df_bracket.fillna(0)
    n_winners = 1
    for r in range(rounds,0,-1):
        cur_col = 'R'+str(r)
        for w in range(n_winners):
            start_i = df_teams.loc[df_teams[cur_col]>w].index[0]
            try:
                end_i = df_teams.loc[df_teams[cur_col]>w+1].index[0]-1
            except IndexError:
                end_i = len(df_teams.index)-1
            if not df_bracket.loc[start_i:end_i, cur_col].max()==1:
                rand = np.random.rand() + w
                win_i = df_teams.loc[df_teams[cur_col]>=rand].index[0]
                win_c = df_teams.columns.tolist().index(cur_col)
                df_bracket.iloc[win_i, :win_c+1] = 1
        n_winners = n_winners*2
    df_bracket = df_bracket.fillna(0)
        
    return df_bracket

In [349]:
def _generate_pool(df_public_picks, pool_size):
    rounds = len(df_public_picks.columns)
    df_public_picks = df_public_picks.cumsum()
    pool = []
    for i in range(pool_size-1):
        df_pool = pd.DataFrame(columns=df_public_picks.columns, index=df_public_picks.index)
        df_pool.fillna(0)
        n_winners = 1
        for r in range(rounds,0,-1):
            cur_col = 'R'+str(r)
            for w in range(n_winners):
                start_i = df_public_picks.loc[df_public_picks[cur_col]>w].index[0]
                try:
                    end_i = df_public_picks.loc[df_public_picks[cur_col]>w+1].index[0]-1
                except IndexError:
                    end_i = len(df_public_picks.index)-1
                if not df_pool.loc[start_i:end_i, cur_col].max()==1:
                    rand = np.random.rand() + w
                    win_i = df_public_picks.loc[df_public_picks[cur_col]>=rand].index[0]
                    win_c = df_public_picks.columns.tolist().index(cur_col)
                    df_pool.iloc[win_i, :win_c+1] = 1
            n_winners = n_winners*2
        df_pool = df_pool.fillna(0)
        pool.append(df_pool)
        
    return pool

In [347]:
def _win_pool_sim(reps, df_public_picks, df_picks, df_predictions, pool_size, rewards):
    wins = 0.0
    rewards = np.array(rewards)
    for i in range(reps):
        pool = [df_picks] + _generate_pool(df_public_picks, pool_size)
        bracket = _generate_bracket(df_predictions)
        winner_count = 0
        cur_max = 0
        for i, p in enumerate(pool):
            correct_picks = p.multiply(bracket).sum().values
            points = np.dot(rewards, correct_picks)
            if points>cur_max:
                winner_index = i
                cur_max = points
                winner_count = 1
            elif points==cur_max:
                winner_count += 1
            else:
                pass
            
        if winner_index==0:
            wins += 1.0/winner_count
        #print ('Number of winners: {0}'.format(str(winner_count)))
        #print ('Winning points: {0}'.format(str(cur_max)))
        #print ('Wins: {0}'.format(str(wins)))
            
    return wins/reps

In [372]:
public_data = [
    [0.6, 0.35, 0.24],
    [0.4, 0.25, 0.23],
    [0.75, 0.4, 0.23],
    [0.25, 0.0, 0.0],
    [0.9, 0.7, 0.25],
    [0.1, 0.04, 0.01],
    [0.85, 0.25, 0.04],
    [0.15, 0.01, 0.0]
]
df_public_picks = pd.DataFrame(public_data, columns=['R1','R2','R3','R4','R5','R6'])
picks_data = [
    [0, 0, 0],
    [1, 1, 1],
    [1, 0, 0],
    [0, 0, 0],
    [1, 1, 0],
    [0, 0, 0],
    [1, 0, 0],
    [0, 0, 0]
]
df_picks = pd.DataFrame(picks_data, columns=['R1','R2','R3','R4','R5','R6'])

In [373]:
pool_size = 12
win_pct = _win_pool_sim(10, df_public_picks, df_picks, df_predictions, pool_size, [1,2,4,8,16,32])
print ('Expected Win Rate: {0}%'.format(str(round(100.0/pool_size,2))))
print ('Actual Win Rate: {0}%'.format(str(round(100*win_pct,2))))

Expected Win Rate: 8.33%
Actual Win Rate: 11.58%
